# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Run the code below for loading the CSV file into Apache Cassandra. Task of data modeling will be followed by the completion of data loading.

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS project1b 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

#### Set Keyspace

In [7]:
session.set_keyspace('project1b')

## Then we have to create tables to serve the proposed queries in the project. Remember, with Apache Cassandra you model the database tables on the queries you want to run respectively.

### Query 1 : Find the artist, song title and song's length in the music app history by sessionId and itemInSession
### We will use condition sessionId = 338 and itemInSession = 4 to run this query
### This query has <font color=green>no</font> requirement in column sorting

In [8]:
# Step 1 : Create table based on the column requirement of the query 1
query = "CREATE TABLE IF NOT EXISTS app_session_history "
query = query + "(session_id int, item_in_session int, artist_name text, song_length decimal, song_title text, PRIMARY KEY (session_id, item_in_session))"
session.execute(query)

In [9]:
# Step 2 : Insert data from CSV file into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_session_history (session_id, item_in_session, artist_name, song_length, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

In [10]:
# Step 3 : Verify the data has been inserted into the table
query = "select session_id, item_in_session, artist_name, song_length, song_title from app_session_history WHERE session_id = 338 AND item_in_session = 4"
rows = session.execute(query)

for row in rows:
    print(row.session_id, row.item_in_session, row.artist_name, row.song_length, row.song_title)

338 4 Faithless 495.3073 Music Matters (Mark Knight Dub)


### Query 2 : Find artist, song title, and full name of user by userid and sessionid
### We will use condition userid = 10 and sessionid = 182 to run this query
### Notice that song title should be sorted by (and <font color=red>only</font> by) itemInSession

In [11]:
# Step 1 : Create table based on the column requirement of the query 2
query = "CREATE TABLE IF NOT EXISTS user_session_history "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
session.execute(query)

In [12]:
# Step 2 : Insert data from CSV file into table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_history (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Step 3 : Verify the data was entered into the table
query = "select user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name from user_session_history WHERE user_id = 10 AND session_id = 182"
rows = session.execute(query)

for row in rows:
    print(row.user_id, row.session_id, row.item_in_session, row.artist_name, row.song_title, row.first_name, row.last_name)

10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
10 182 1 Three Drives Greece 2000 Sylvie Cruz
10 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 : Find users' full name by song title they have listened to
### We will use condition song title = 'All Hands Against His Own' to answer this question
### This query has <font color=green>no</font> requirement in column sorting

In [14]:
# Step 1 : Create table based on the column requirement of the query 3
query = "CREATE TABLE IF NOT EXISTS app_song_audience "
query = query + "(song_title text, user_id int, session_id int, item_in_session int, first_name text, last_name text, PRIMARY KEY (song_title, user_id, session_id, item_in_session))"
session.execute(query)

In [15]:
# Step 2 : Insert data from CSV file into table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_song_audience (song_title, user_id, session_id, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))

In [16]:
# Step 3 : Verify the data was entered into the table
query = "select song_title, first_name, last_name from app_song_audience WHERE song_title = 'All Hands Against His Own'"
rows = session.execute(query)

for row in rows:
    print(row.song_title, row.first_name, row.last_name)

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


### Drop the tables and keyspace before closing out the sessions

In [17]:
session.execute("DROP TABLE app_session_history")
session.execute("DROP TABLE user_session_history")
session.execute("DROP TABLE app_song_audience")
session.execute("""DROP KEYSPACE IF EXISTS project1b""")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()